### Librerías

In [1]:
# Tratamiento de datos
import pandas as pd
import numpy as np

# Gráficos
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

# Estadística 
import scipy

import warnings

In [ ]:
# Dependencias instaladas en la terminal:
# - pip install pandas pyarrow
# - pip install pandas fastparquet

### Carga de datos

In [2]:
df = pd.read_parquet(r'C:\Users\nuria\OneDrive\Escritorio\Data Science\DS102024\3-MachineLearning\9-ML\train.snappy', engine='pyarrow')
df.head()

,descripcion
0,**La observación** 367 describe una seta de *...
1,**La observación** 5123 describe una seta de ...
2,**La observación** 2864 describe una seta de ...
3,**La observación** 4674 describe una seta de ...
4,**La observación** 1210 describe una seta de ...


In [3]:
df.tail()

,descripcion
4598,**La observación** 1028 describe una seta de ...
4599,**La observación** 5591 describe una seta de ...
4600,**La observación** 2053 describe una seta de ...
4601,**La observación** 4604 describe una seta de ...
4602,**La observación** 4790 describe una seta de ...


In [4]:
print(df.shape)
print('-----------------------------------------------------------------------------------')
print(df.columns)
print('-----------------------------------------------------------------------------------')
print(df.info())
print('-----------------------------------------------------------------------------------')

(4603, 1)
-----------------------------------------------------------------------------------
Index(['descripcion'], dtype='object')
-----------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4603 entries, 0 to 4602
Data columns (total 1 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   descripcion  4603 non-null   object
dtypes: object(1)
memory usage: 36.1+ KB
None
-----------------------------------------------------------------------------------


In [5]:
df.describe()

,descripcion
count,4603
unique,4603
top,**La observación** 367 describe una seta de *...
freq,1


In [6]:
# Comprobamos que no tenga valores duplicados
df[df.duplicated(keep = False)] 

,descripcion


In [7]:
summary_df = pd.DataFrame(columns=[
    'Nombre de la variable', 
    'Tipo de dato', 
    'Tipo de variable', 
    'Cardinalidad', 
    'Distribución', 
    'Valores faltantes (cantidad)', 
    'Valores faltantes (%)', 
    'Outliers (cantidad)', 
    'Outliers (%)', 
    'Rango de valores', 
    'Moda', 
    'Media', 
    'Mediana', 
    'Asimetría', 
    'Curtosis', 
    'Resultados de pruebas de normalidad'
])

def summarize_variable(col):
    data_type = col.dtype
    variable_type = 'numérica' if np.issubdtype(data_type, np.number) else 'categórica'
    cardinality = col.nunique()
    normality_test_result = 'N/A'

    if np.issubdtype(data_type, np.number):
        # Test de K-S
        d_statistic, p_value = stats.kstest(col.dropna(), 'norm', args=(col.mean(), col.std()))
        normality_test_result = 'Normal' if p_value > 0.05 else 'No normal'
        distribution = 'Gaussiana' if normality_test_result == 'Normal' else 'No gaussiana'
    else:
        distribution = 'Categórica'
    
    # Valores faltantes
    missing_count = col.isnull().sum()
    missing_percentage = missing_count / len(col) * 100
    
    # Outliers
    if np.issubdtype(data_type, np.number):
        q1 = col.quantile(0.25)
        q3 = col.quantile(0.75)
        iqr = q3 - q1
        lower_bound = q1 - 1.5 * iqr
        upper_bound = q3 + 1.5 * iqr
        outliers = col[(col < lower_bound) | (col > upper_bound)]
        outliers_count = outliers.count()
        outliers_percentage = outliers_count / len(col) * 100
    else:
        outliers_count = 0
        outliers_percentage = 0

    value_range = f"{col.min()} - {col.max()}" if np.issubdtype(data_type, np.number) else 'N/A'
    
    mode = col.mode()[0] if variable_type == 'categórica' else col.mode().values[0]
    mean = col.mean() if variable_type == 'numérica' else 'N/A'
    median = col.median() if variable_type == 'numérica' else 'N/A'
    skewness = col.skew() if variable_type == 'numérica' else 'N/A'
    kurtosis = col.kurtosis() if variable_type == 'numérica' else 'N/A'

    summary_df.loc[len(summary_df)] = [
        col.name, 
        data_type, 
        variable_type, 
        cardinality, 
        distribution, 
        missing_count, 
        missing_percentage, 
        outliers_count, 
        outliers_percentage, 
        value_range, 
        mode, 
        mean, 
        median, 
        skewness, 
        kurtosis, 
        normality_test_result
    ]

# Resumir cada variable en el DataFrame
for column in df.columns:
    summarize_variable(df[column])

# Configurar el DataFrame para mejor visualización
summary_df.set_index('Nombre de la variable', inplace=True)
pd.set_option('display.max_columns', None)  # Mostrar todas las columnas
pd.set_option('display.expand_frame_repr', False)  # Evitar el truncado de filas

# Mostrar el resumen en formato DataFrame
import IPython.display as display
display.display(summary_df)

,Tipo de dato,Tipo de variable,Cardinalidad,Distribución,Valores faltantes (cantidad),Valores faltantes (%),Outliers (cantidad),Outliers (%),Rango de valores,Moda,Media,Mediana,Asimetría,Curtosis,Resultados de pruebas de normalidad
Nombre de la variable,,,,,,,,,,,,,,,
descripcion,object,categórica,4603,Categórica,0,0.0,0,0,N/A,**La observación** 0 describe una seta de **c...,N/A,N/A,N/A,N/A,N/A


## Tratamiento de datos

In [8]:
df.head()

,descripcion
0,**La observación** 367 describe una seta de *...
1,**La observación** 5123 describe una seta de ...
2,**La observación** 2864 describe una seta de ...
3,**La observación** 4674 describe una seta de ...
4,**La observación** 1210 describe una seta de ...


In [95]:
split_data = df['descripcion'].str.split(r'\*\*', expand=True)
split_data.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48
0,,La observación,367 describe una seta de,clase,edible. Tiene una,forma de sombrero,bell con una,superficie,scaly,y un color,yellow.,Presenta,moretones y tiene un,olor,almond.,Las branquias son de tipo,free con una,separación,close,y un tamaño,broad.,El color de las branquias es,black.,La forma del tallo,es enlarging y la,raíz del tallo,es club.,La superficie del tallo por encima del anillo,es smooth,y por debajo del anillo,es smooth. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,pendant,y el color de la impresión de esporas,es brown. Se encuentra en una,población,scattered y su,hábitat es,grasses.
1,,La observación,5123 describe una seta de,clase,poisonous. Tiene una,forma de sombrero,convex con una,superficie,smooth,y un color,red.,No presenta,moretones y tiene un,olor,foul.,Las branquias son de tipo,free con una,separación,close,y un tamaño,narrow.,El color de las branquias es,buff.,La forma del tallo,es tapering y la,raíz del tallo,es ?.,La superficie del tallo por encima del anillo,es silky,y por debajo del anillo,es silky. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es pink.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,evanescent,y el color de la impresión de esporas,es white. Se encuentra en una,población,several y su,hábitat es,paths.
2,,La observación,2864 describe una seta de,clase,edible. Tiene una,forma de sombrero,convex con una,superficie,fibrous,y un color,green.,Presenta,moretones y tiene un,olor,null.,Las branquias son de tipo,free con una,separación,close,y un tamaño,broad.,El color de las branquias es,pink.,La forma del tallo,es tapering y la,raíz del tallo,es bulbous.,La superficie del tallo por encima del anillo,es smooth,y por debajo del anillo,es smooth. El,color del tallo por encima del anillo,es pink,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,pendant,y el color de la impresión de esporas,es brown. Se encuentra en una,población,several y su,hábitat es,wood.
3,,La observación,4674 describe una seta de,clase,poisonous. Tiene una,forma de sombrero,flat con una,superficie,smooth,y un color,white.,Presenta,moretones y tiene un,olor,foul.,Las branquias son de tipo,free con una,separación,close,y un tamaño,broad.,El color de las branquias es,pink.,La forma del tallo,es tapering y la,raíz del tallo,es bulbous.,La superficie del tallo por encima del anillo,es fibrous,y por debajo del anillo,es fibrous. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,pendant,y el color de la impresión de esporas,es chocolate. Se encuentra en una,población,scattered y su,hábitat es,grasses.
4,,La observación,1210 describe una seta de,clase,edible. Tiene una,forma de sombrero,convex con una,superficie,smooth,y un color,white.,No presenta,moretones y tiene un,olor,null.,Las branquias son de tipo,free con una,separación,crowded,y un tamaño,broad.,El color de las branquias es,black.,La forma del tallo,es tapering y la,raíz del tallo,es equal.,La superficie del tallo por encima del anillo,es fibrous,y por debajo del anillo,es fibrous. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,evanescent,y el color de la impresión de esporas,es black. Se encuentra en una,población,abundant y su,hábitat es,grasses.


In [29]:
# Renombrar columnas
df1= split_data.rename(columns={2: 'id', 4: 'clase', 6: 'forma sombrero', 8: 'superficie', 10: 'color', 11:'moretones', 14: 'olor', 16: 'tipo branquias', 18: 'separacion branquias', 20: 'tamaño', 22: 'color branquias', 24: 'forma tallo', 26: 'raiz tallo', 28: 'superficie tallo encima anillo', 30: 'superficie tallo debajo anillo', 32: 'color tallo encima anillo', 34: 'color tallo debajo anillo', 36: 'tipo velo', 38: 'color velo / nº anillos', 42: 'tipo anillo', 44: 'color impresion esporas', 46: 'poblacion', 48: 'habitat'})
df1

,0,1,id,3,clase,5,forma sombrero,7,superficie,9,color,moretones,12,13,olor,15,tipo branquias,17,separacion branquias,19,tamaño,21,color branquias,23,forma tallo,25,raiz tallo,27,superficie tallo encima anillo,29,superficie tallo debajo anillo,31,color tallo encima anillo,33,color tallo debajo anillo,35,tipo velo,37,color velo / nº anillos,39,40,41,tipo anillo,43,color impresion esporas,45,poblacion,47,habitat
0,,La observación,367 describe una seta de,clase,edible. Tiene una,forma de sombrero,bell con una,superficie,scaly,y un color,yellow.,Presenta,moretones y tiene un,olor,almond.,Las branquias son de tipo,free con una,separación,close,y un tamaño,broad.,El color de las branquias es,black.,La forma del tallo,es enlarging y la,raíz del tallo,es club.,La superficie del tallo por encima del anillo,es smooth,y por debajo del anillo,es smooth. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,pendant,y el color de la impresión de esporas,es brown. Se encuentra en una,población,scattered y su,hábitat es,grasses.
1,,La observación,5123 describe una seta de,clase,poisonous. Tiene una,forma de sombrero,convex con una,superficie,smooth,y un color,red.,No presenta,moretones y tiene un,olor,foul.,Las branquias son de tipo,free con una,separación,close,y un tamaño,narrow.,El color de las branquias es,buff.,La forma del tallo,es tapering y la,raíz del tallo,es ?.,La superficie del tallo por encima del anillo,es silky,y por debajo del anillo,es silky. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es pink.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,evanescent,y el color de la impresión de esporas,es white. Se encuentra en una,población,several y su,hábitat es,paths.
2,,La observación,2864 describe una seta de,clase,edible. Tiene una,forma de sombrero,convex con una,superficie,fibrous,y un color,green.,Presenta,moretones y tiene un,olor,null.,Las branquias son de tipo,free con una,separación,close,y un tamaño,broad.,El color de las branquias es,pink.,La forma del tallo,es tapering y la,raíz del tallo,es bulbous.,La superficie del tallo por encima del anillo,es smooth,y por debajo del anillo,es smooth. El,color del tallo por encima del anillo,es pink,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,pendant,y el color de la impresión de esporas,es brown. Se encuentra en una,población,several y su,hábitat es,wood.
3,,La observación,4674 describe una seta de,clase,poisonous. Tiene una,forma de sombrero,flat con una,superficie,smooth,y un color,white.,Presenta,moretones y tiene un,olor,foul.,Las branquias son de tipo,free con una,separación,close,y un tamaño,broad.,El color de las branquias es,pink.,La forma del tallo,es tapering y la,raíz del tallo,es bulbous.,La superficie del tallo por encima del anillo,es fibrous,y por debajo del anillo,es fibrous. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,pendant,y el color de la impresión de esporas,es chocolate. Se encuentra en una,población,scattered y su,hábitat es,grasses.
4,,La observación,1210 describe una seta de,clase,edible. Tiene una,forma de sombrero,convex con una,superficie,smooth,y un color,white.,No presenta,moretones y tiene un,olor,null.,Las branquias son de tipo,free con una,separación,crowded,y un tamaño,broad.,El color de las branquias es,black.,La forma del tallo,es tapering y la,raíz del tallo,es equal.,La superficie del tallo por encima del anillo,es fibrous,y por debajo del anillo,es fibrous. El,color del tallo por encima del anillo,es white,y por debajo del anillo,es white.,El tipo de velo,es partial con un,color de velo,white. Tiene one,anillo(s),",",de tipo,evanescent,y el color de la impresión

In [ ]:
# Eliminar las columnas no necesarias
df2 = df1.drop([1,3,5,7,9,12,13,15,17,19,21,23,25,27,29,31,33,35,37,39,40,41,43,45,47], axis = 1)
df2

,0,id,clase,forma sombrero,superficie,color,moretones,olor,tipo branquias,separacion branquias,tamaño,color branquias,forma tallo,raiz tallo,superficie tallo encima anillo,superficie tallo debajo anillo,color tallo encima anillo,color tallo debajo anillo,tipo velo,color velo / nº anillos,tipo anillo,color impresion esporas,poblacion,habitat
0,,367 describe una seta de,edible. Tiene una,bell con una,scaly,yellow.,Presenta,almond.,free con una,close,broad.,black.,es enlarging y la,es club.,es smooth,es smooth. El,es white,es white.,es partial con un,white. Tiene one,pendant,es brown. Se encuentra en una,scattered y su,grasses.
1,,5123 describe una seta de,poisonous. Tiene una,convex con una,smooth,red.,No presenta,foul.,free con una,close,narrow.,buff.,es tapering y la,es ?.,es silky,es silky. El,es white,es pink.,es partial con un,white. Tiene one,evanescent,es white. Se encuentra en una,several y su,paths.
2,,2864 describe una seta de,edible. Tiene una,convex con una,fibrous,green.,Presenta,null.,free con una,close,broad.,pink.,es tapering y la,es bulbous.,es smooth,es smooth. El,es pink,es white.,es partial con un,white. Tiene one,pendant,es brown. Se encuentra en una,several y su,wood.
3,,4674 describe una seta de,poisonous. Tiene una,flat con una,smooth,white.,Presenta,foul.,free con una,close,broad.,pink.,es tapering y la,es bulbous.,es fibrous,es fibrous. El,es white,es white.,es partial con un,white. Tiene one,pendant,es chocolate. Se encuentra en una,scattered y su,grasses.
4,,1210 describe una seta de,edible. Tiene una,convex con una,smooth,white.,No presenta,null.,free con una,crowded,broad.,black.,es tapering y la,es equal.,es fibrous,es fibrous. El,es white,es white.,es partial con un,white. Tiene one,evanescent,es black. Se encuentra en una,abundant y su,grasses.
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4598,,1028 describe una seta de,edible. Tiene una,convex con una,fibrous,brown.,Presenta,null.,free con una,close,broad.,purple.,es tapering y la,es bulbous.,es smooth,es smooth. El,es gray,es gray.,es partial con un,white. Tiene one,pendant,es black. Se encuentra en una,several y su,wood.
4599,,5591 describe una seta de,poisonous. Tiene una,convex con una,smooth,red.,No presenta,fishy.,free con una,close,narrow.,buff.,es tapering y la,es ?.,es smooth,es silky. El,es pink,es white.,es partial con un,white. Tiene one,evanescent,es white. Se encuentra en una,several y su,wood.
4600,,2053 describe una seta de,edible. Tiene una,convex con una,fibrous,red.,Presenta,null.,free con una,close,broad.,brown.,es tapering y la,es bulbous.,es smooth,es smooth. El,es white,es gray.,es partial con un,white. Tiene one,pendant,es brown. Se encuentra en una,solitary y su,wood.
4601,,4604 describe una seta de,poisonous. Tiene una,convex con una,scaly,brown.,No presenta,foul.,free con una,close,narrow.,buff.,es tapering y la,es ?.,es smooth,es smooth. El,es pink,es pink.,es partial con un,white. Tiene one,evanescent,es white. Se encuentra en una,several y su,paths.


In [39]:
df2['id'] = df2['id'].str.replace('describe una seta de', '', regex=False)

In [40]:
df2['clase'] = df2['clase'].str.replace('Tiene una', '', regex=False)

In [70]:
df2['forma sombrero'] = df2['forma sombrero'].str.replace('con una', '', regex=False)
df2['tipo branquias'] = df2['tipo branquias'].str.replace('con una', '', regex=False)
df2['forma tallo'] = df2['forma tallo'].str.replace('es', '', regex=False)
df2['forma tallo'] = df2['forma tallo'].str.replace('y la', '', regex=False)
df2['raiz tallo'] = df2['raiz tallo'].str.replace('es', '', regex=False)
df2['superficie tallo encima anillo'] = df2['superficie tallo encima anillo'].str.replace('es', '', regex=False)
df2['superficie tallo debajo anillo'] = df2['superficie tallo debajo anillo'].str.replace('es', '', regex=False)
df2['superficie tallo debajo anillo'] = df2['superficie tallo debajo anillo'].str.replace('El', '', regex=False)
df2['color tallo encima anillo'] = df2['color tallo encima anillo'].str.replace('es', '', regex=False)
df2['color tallo debajo anillo'] = df2['color tallo debajo anillo'].str.replace('es', '', regex=False)
df2['tipo velo'] = df2['tipo velo'].str.replace('es', '', regex=False)
df2['tipo velo'] = df2['tipo velo'].str.replace('con un', '', regex=False)
df2['color impresion esporas'] = df2['color impresion esporas'].str.replace('es', '', regex=False)
df2['color impresion esporas'] = df2['color impresion esporas'].str.replace('Se encuentra en una', '', regex=False)
df2['poblacion'] = df2['poblacion'].str.replace('y su', '', regex=False)

In [75]:
# Duplicra una columna
df2['numero anillos'] = df2['color velo / nº anillos']
df2.head(3)

,0,id,clase,forma sombrero,superficie,color,moretones,olor,tipo branquias,separacion branquias,tamaño,color branquias,forma tallo,raiz tallo,superficie tallo encima anillo,superficie tallo debajo anillo,color tallo encima anillo,color tallo debajo anillo,tipo velo,color velo / nº anillos,tipo anillo,color impresion esporas,poblacion,habitat,numero anillos
0,,367,edible.,bell,scaly,yellow.,Presenta,almond.,free,close,broad.,black.,enlarging,club.,smooth,smooth.,white,white.,partial,white. Tiene one,pendant,brown.,scattered,grasses.,white. Tiene one
1,,5123,poisonous.,convex,smooth,red.,No presenta,foul.,free,close,narrow.,buff.,tapering,?.,silky,silky.,white,pink.,partial,white. Tiene one,evanescent,white.,several,paths.,white. Tiene one
2,,2864,edible.,convex,fibrous,green.,Presenta,null.,free,close,broad.,pink.,tapering,bulbous.,smooth,smooth.,pink,white.,partial,white. Tiene one,pendant,brown.,several,wood.,white. Tiene one


In [92]:
df2= df2.rename(columns={'color velo / nº anillos': 'color velo'})

In [90]:
df2['color velo'] = df2['color velo'].str.replace('Tiene one', '', regex=False)
df2['numero anillos'] = df2['numero anillos'].str.replace('white.', '', regex=False)
df2['numero anillos'] = df2['numero anillos'].str.replace('Tiene', '', regex=False)

In [99]:
df2.head(6)

,0,id,clase,forma sombrero,superficie,color,moretones,olor,tipo branquias,separacion branquias,tamaño,color branquias,forma tallo,raiz tallo,superficie tallo encima anillo,superficie tallo debajo anillo,color tallo encima anillo,color tallo debajo anillo,tipo velo,color velo,tipo anillo,color impresion esporas,poblacion,habitat,numero anillos
0,,367,edible.,bell,scaly,yellow.,Presenta,almond.,free,close,broad.,black.,enlarging,club.,smooth,smooth.,white,white.,partial,white.,pendant,brown.,scattered,grasses.,. one
1,,5123,poisonous.,convex,smooth,red.,No presenta,foul.,free,close,narrow.,buff.,tapering,?.,silky,silky.,white,pink.,partial,white.,evanescent,white.,several,paths.,. one
2,,2864,edible.,convex,fibrous,green.,Presenta,null.,free,close,broad.,pink.,tapering,bulbous.,smooth,smooth.,pink,white.,partial,white.,pendant,brown.,several,wood.,. one
3,,4674,poisonous.,flat,smooth,white.,Presenta,foul.,free,close,broad.,pink.,tapering,bulbous.,fibrous,fibrous.,white,white.,partial,white.,pendant,chocolate.,scattered,grasses.,. one
4,,1210,edible.,convex,smooth,white.,No presenta,null.,free,crowded,broad.,black.,tapering,equal.,fibrous,fibrous.,white,white.,partial,white.,evanescent,black.,abundant,grasses.,. one
5,,5381,poisonous.,flat,smooth,red.,No presenta,spicy.,free,close,narrow.,buff.,tapering,?.,silky,silky.,pink,white.,partial,white.,evanescent,white.,several,leaves.,. one


In [94]:
df2.shape 

(4603, 25)

## Predicción

In [100]:
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

In [101]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
for column in df2.select_dtypes(include=['object']).columns:
    df2[column] = label_encoder.fit_transform(df2[column])

In [103]:
# Seleccionar solo las columnas numéricas para escalar
numeric_cols = df2.select_dtypes(include=['int64', 'float64']).columns

# Normalizar las características numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df2[numeric_cols])

ValueError: at least one array or dtype is required

In [102]:
# Paso 1: Codificación de datos categóricos
df_encoded = pd.get_dummies(df2)

# Paso 2: Preprocesar datos
scaler = StandardScaler()
X_scaled = scaler.fit_transform(df_encoded)

# Paso 3: Aplicar el modelo de clustering
optimal_k = 4  
kmeans = KMeans(n_clusters=optimal_k, random_state=42)
kmeans.fit(X_scaled)
 
# Obtener las etiquetas de los clústeres
df2['cluster'] = kmeans.labels_

TypeError: Feature names are only supported if all input features have string names, but your input has ['int', 'str'] as feature name / column name types. If you want feature names to be stored and validated, you must convert them all to strings, by using X.columns = X.columns.astype(str) for example. Otherwise you can remove feature / column names from your input data, or convert them all to a non-string data type.